In [0]:
dbutils.fs.ls('/mnt/bronze/SalesLT')

[FileInfo(path='dbfs:/mnt/bronze/SalesLT/Address/', name='Address/', size=0, modificationTime=1698204157000),
 FileInfo(path='dbfs:/mnt/bronze/SalesLT/Customer/', name='Customer/', size=0, modificationTime=1698204157000),
 FileInfo(path='dbfs:/mnt/bronze/SalesLT/CustomerAddress/', name='CustomerAddress/', size=0, modificationTime=1698204155000),
 FileInfo(path='dbfs:/mnt/bronze/SalesLT/Product/', name='Product/', size=0, modificationTime=1698204157000),
 FileInfo(path='dbfs:/mnt/bronze/SalesLT/ProductCategory/', name='ProductCategory/', size=0, modificationTime=1698204156000),
 FileInfo(path='dbfs:/mnt/bronze/SalesLT/ProductDescription/', name='ProductDescription/', size=0, modificationTime=1698204155000),
 FileInfo(path='dbfs:/mnt/bronze/SalesLT/ProductModel/', name='ProductModel/', size=0, modificationTime=1698204160000),
 FileInfo(path='dbfs:/mnt/bronze/SalesLT/ProductModelProductDescription/', name='ProductModelProductDescription/', size=0, modificationTime=1698204155000),
 FileInf

## Transforming date column in Address table

In [0]:
address_tbl_path = '/mnt/bronze/SalesLT/Address/Address.parquet'
address_df = spark.read.format("parquet").load(address_tbl_path)

In [0]:
address_df.show(10)

+---------+-------------------+------------+--------+----------------+-------------+----------+--------------------+-------------------+
|AddressID|       AddressLine1|AddressLine2|    City|   StateProvince|CountryRegion|PostalCode|             rowguid|       ModifiedDate|
+---------+-------------------+------------+--------+----------------+-------------+----------+--------------------+-------------------+
|        9|  8713 Yosemite Ct.|        NULL| Bothell|      Washington|United States|     98011|268af621-76d7-4c7...|2006-07-01 00:00:00|
|       11|1318 Lasalle Street|        NULL| Bothell|      Washington|United States|     98011|981b3303-aca2-49c...|2007-04-01 00:00:00|
|       25|   9178 Jumping St.|        NULL|  Dallas|           Texas|United States|     75201|c8df3bd9-48f0-465...|2006-09-01 00:00:00|
|       28|   9228 Via Del Sol|        NULL| Phoenix|         Arizona|United States|     85004|12ae5ee1-fc3e-468...|2005-09-01 00:00:00|
|       32|  26910 Indela Road|        NU

In [0]:
from pyspark.sql.functions import from_utc_timestamp, date_format
from pyspark.sql.types import TimestampType

In [0]:
address_df = address_df.withColumn("ModifiedDate", date_format(from_utc_timestamp(address_df["ModifiedDate"].cast(TimestampType()), "UTC"), "yyyy-MM-dd"))

In [0]:
address_df.show(10)

+---------+-------------------+------------+--------+----------------+-------------+----------+--------------------+------------+
|AddressID|       AddressLine1|AddressLine2|    City|   StateProvince|CountryRegion|PostalCode|             rowguid|ModifiedDate|
+---------+-------------------+------------+--------+----------------+-------------+----------+--------------------+------------+
|        9|  8713 Yosemite Ct.|        NULL| Bothell|      Washington|United States|     98011|268af621-76d7-4c7...|  2006-07-01|
|       11|1318 Lasalle Street|        NULL| Bothell|      Washington|United States|     98011|981b3303-aca2-49c...|  2007-04-01|
|       25|   9178 Jumping St.|        NULL|  Dallas|           Texas|United States|     75201|c8df3bd9-48f0-465...|  2006-09-01|
|       28|   9228 Via Del Sol|        NULL| Phoenix|         Arizona|United States|     85004|12ae5ee1-fc3e-468...|  2005-09-01|
|       32|  26910 Indela Road|        NULL|Montreal|          Quebec|       Canada|   H1Y

## Doing transformation for all tables

In [0]:
table_names = []

for tbl in dbutils.fs.ls('mnt/bronze/SalesLT'):
    table_names.append(tbl.name.split('/')[0])

In [0]:
table_names

['Address',
 'Customer',
 'CustomerAddress',
 'Product',
 'ProductCategory',
 'ProductDescription',
 'ProductModel',
 'ProductModelProductDescription',
 'SalesOrderDetail',
 'SalesOrderHeader']

In [0]:
for t in table_names:
    path = '/mnt/bronze/SalesLT/' + t + '/' + t + '.parquet'
    df = spark.read.format('parquet').load(path)
    cols = df.columns

    for col in cols:
        if "Date" in col or "date" in col:
            df = df.withColumn (col, date_format(from_utc_timestamp(df[col].cast(TimestampType()), "UTC"), "yyyy-MM-dd"))
    
    output_path = '/mnt/silver/SalesLT/' + t + '/'
    df.write.format('delta').mode("overwrite").save(output_path)

In [0]:
display(df)

SalesOrderID RevisionNumber OrderDate DueDate ShipDate Status OnlineOrderFlag SalesOrderNumber PurchaseOrderNumber AccountNumber CustomerID ShipToAddressID BillToAddressID ShipMethod CreditCardApprovalCode SubTotal TaxAmt Freight TotalDue Comment rowguid ModifiedDate 71774 2 2008-06-01 2008-06-13 2008-06-08 5 false SO71774 PO348186287 10-4020-000609 29847 1092 1092 CARGO TRANSPORT 5 null 880.3484 70.4279 22.0087 972.7850 null 89e42cdc-8506-48a2-b89b-eb3e64e3554e 2008-06-08 71776 2 2008-06-01 2008-06-13 2008-06-08 5 false SO71776 PO19952192051 10-4020-000106 30072 640 640 CARGO TRANSPORT 5 null 78.8100 6.3048 1.9703 87.0851 null 8a3448c5-e677-4158-a29b-dd33069be0b0 2008-06-08 71780 2 2008-06-01 2008-06-13 2008-06-08 5 false SO71780 PO19604173239 10-4020-000340 30113 653 653 CARGO TRANSPORT 5 null 38418.6895 3073.4952 960.4672 42452.6519 null a47665d2-7ac9-4cf3-8a8b-2a3883554284 2008-06-08 71782 2 2008-06-01 2008-06-13 2008-06-08 5 false SO71782 PO19372114749 10-4020-000582 29485 1086 1086 CARGO TRANSPORT 5 null 39785.3304 3182.8264 994.6333 43962.7901 null f1be45a5-5c57-4a50-93c6-5f8be44cb7cb 2008-06-08 71783 2 2008-06-01 2008-06-13 2008-06-08 5 false SO71783 PO19343113609 10-4020-000024 29957 992 992 CARGO TRANSPORT 5 null 83858.4261 6708.6741 2096.4607 92663.5609 null 7db2329e-6446-42a8-8915-9c8370b68ed8 2008-06-08 71784 2 2008-06-01 2008-06-13 2008-06-08 5 false SO71784 PO19285135919 10-4020-000448 29736 659 659 CARGO TRANSPORT 5 null 108561.8317 8684.9465 2714.0458 119960.8240 null ca31f324-2c32-4f8d-95eb-596e7f343027 2008-06-08 71796 2 2008-06-01 2008-06-13 2008-06-08 5 false SO71796 PO17052159664 10-4020-000420 29660 1058 1058 CARGO TRANSPORT 5 null 57634.6342 4610.7707 1440.8659 63686.2708 null 917ef5ba-f32d-4563-8588-66db0bcdc846 2008-06-08 71797 2 2008-06-01 2008-06-13 2008-06-08 5 false SO71797 PO16501134889 10-4020-000142 29796 642 642 CARGO TRANSPORT 5 null 78029.6898 6242.3752 1950.7422 86222.8072 null bb3fee84-c8bf-4dd2-bcca-675ab6a11c38 2008-06-08 71815 2 2008-06-01 2008-06-13 2008-06-08 5 false SO71815 PO13021155785 10-4020-000276 30089 1034 1034 CARGO TRANSPORT 5 null 1141.5782 91.3263 28.5395 1261.4440 null 2aa5f39b-1096-4a4b-b17b-f10504a397ce 2008-06-08 71816 2 2008-06-01 2008-06-13 2008-06-08 5 false SO71816 PO12992180445 10-4020-000295 30027 1038 1038 CARGO TRANSPORT 5 null 3398.1659 271.8533 84.9541 3754.9733 null e3c189e7-98de-4c40-b6c2-0d1d13f9bb33 2008-06-08 71831 2 2008-06-01 2008-06-13 2008-06-08 5 false SO71831 PO10295111084 10-4020-000322 30019 652 652 CARGO TRANSPORT 5 null 2016.3408 161.3073 50.4085 2228.0566 null 625d76fc-c26f-4149-bf24-939fb2bccd77 2008-06-08 71832 2 2008-06-01 2008-06-13 2008-06-08 5 false SO71832 PO10353140756 10-4020-000088 29922 639 639 CARGO TRANSPORT 5 null 35775.2113 2862.0169 894.3803 39531.6085 null addb8620-432a-456e-8470-1bedd4bc3457 2008-06-08 71845 2 2008-06-01 2008-06-13 2008-06-08 5 false SO71845 PO2697119362 10-4020-000187 29938 1020 1020 CARGO TRANSPORT 5 null 41622.0511 3329.7641 1040.5513 45992.3665 null e68f7ee9-c581-45cd-9c4f-386aeda74d84 2008-06-08 71846 2 2008-06-01 2008-06-13 2008-06-08 5 false SO71846 PO2378131604 10-4020-000635 30102 669 669 CARGO TRANSPORT 5 null 2453.7645 196.3012 61.3441 2711.4098 null a86d90ad-d1c0-440d-9a57-5b763bf18234 2008-06-08 71856 2 2008-06-01 2008-06-13 2008-06-08 5 false SO71856 PO16530177647 10-4020-000601 30033 1090 1090 CARGO TRANSPORT 5 null 602.1946 48.1756 15.0549 665.4251 null 05fee073-0640-4a3c-914d-fe4ae6da3d43 2008-06-08 71858 2 2008-06-01 2008-06-13 2008-06-08 5 false SO71858 PO16153112278 10-4020-000186 29653 1019 1019 CARGO TRANSPORT 5 null 13823.7083 1105.8967 345.5927 15275.1977 null 5ef091e1-a0af-437d-85ed-0b557c7923f7 2008-06-08 71863 2 2008-06-01 2008-06-13 2008-06-08 5 false SO71863 PO16124166561 10-4020-000649 29975 1098 1098 CARGO TRANSPORT 5 null 3324.2759 265.9421 83.1069 3673.3249 null 3ed03b56-a4bf-4872-9471-bc6c7893eab7 2008-06-08 71867 2 2008-06-01 2008-06-13 2008-06-08 5 false SO71867 PO130501115